# Producing Bottles

See textfile for explanation

In [ ]:
import scipy.stats as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Input

probability functions and parameters, number of experiments, number of bottles produced

In [ ]:
n = 1000 #1000 experiments
m = 100 #100 bottles

lambda1 = 1 #arrival rate
lambda2 = 1.3 #service rate

max_stuck = 3 #how many bottles may be in second stage before machine breaks
# If lambda1 >> lambda2, probably you will break the system a lot

# Random draws for arrival and service

One row = one scenario

In [ ]:
# How long until next arrival
arrivals = st.expon.rvs(size=(n,m), scale = lambda1)
arrivals = arrivals.cumsum(axis=1)
arrivals = pd.DataFrame(arrivals)

# How long until service
service = st.expon.rvs(size=(n,m), scale = lambda2)
service = pd.DataFrame(service)

# Function

In [ ]:
def process(arrivals, service, buffer):
    print("Starting calculation for buffer " + str(buffer))
    finish = pd.DataFrame(np.zeros(arrivals.shape))
    occupied = pd.DataFrame(np.zeros(arrivals.shape))
    stuck = pd.DataFrame(np.zeros(arrivals.shape))
    arrival_at_two = pd.DataFrame(np.zeros(arrivals.shape))

    for t in range(arrivals.shape[1]):
        
        if t==0:
            arrival_at_two.iloc[:,t] = arrivals.iloc[:,t]
            finish.iloc[:,t] = arrival_at_two.iloc[:,t]+service.iloc[:,t]

        else:
            occupied.iloc[:,t] = (finish.iloc[:,:t].apply(lambda x: x>arrivals.iloc[:,t]) & # not finished
                                  arrival_at_two.iloc[:,:t].apply(lambda x: x<arrivals.iloc[:,t])).any(axis=1) # but has arrived
            
            stuck.iloc[:,t] = (finish.iloc[:,:t].apply(lambda x: x>arrivals.iloc[:,t]) & # not finished
                               arrival_at_two.iloc[:,:t].apply(lambda x: x<arrivals.iloc[:,t])).sum(axis=1) # but has arrived
            
            arrival_at_two.iloc[:,t] = arrivals.iloc[:,t].values+buffer*occupied.iloc[:,t].values

            finish.iloc[:,t] = arrival_at_two.iloc[:,t]+service.iloc[:,t]
           
    print("Finished.")
    
    return arrival_at_two, finish, occupied, stuck

# Choose some buffer times, run the simulation

In [ ]:
buffers = [0,1,2,5,10,15,20,35,50,60,75,100,150]

results = [process(arrivals, service, buffer) for buffer in buffers]

# Breakdowns

In [ ]:
breakdowns = [(results[i][3].max(axis=1)>max_stuck).sum() for i in range(len(buffers))]

In [ ]:
breakdowns

In [ ]:
plt.plot(buffers, breakdowns)

# Production rate

In [ ]:
mean_process_time = [results[i][1].max(axis=1).mean() for i in range(len(buffers))]
mean_process_time

In [ ]:
plt.plot(buffers, mean_process_time)